# Step 1: Read & Process Concept Reports
We first read the transcript text files, grouping together the files in the "Free" group as one set, and the "D-C" group as another set.

To avoid processing errors, we first remove all non-UTF characters in the text (such as distinct opening and closing quotes)
and replace them with corresponding UTF characters. These transcripts also come with the transcriber's observations, such as [laughter], or [silent]. We remove all such observations so that we are only looking at team members' speech.

Finally we remove all stop words

In [1]:
import re
# Locations of input files. These files have been anonymized, and the files themselves will not be available
# in this folder, to preserve the privacy of the teams.
textPathFree = "../../../private/brainstorming_viz_data/free_group/"
textPathDC = "../../../private/brainstorming_viz_data/d-c_group/"

# Teamwise transcript (of divergent processes) of the "Free Brainstorming" group
fileNamesFree = ["concept_reflection.txt"]

# Teamwise transcript (of divergent processes) of the "Divergent-Convergent (D-C) Brainstorming" group
fileNamesDC = ["concept_reflection_d-c.txt"]

def openFiles(fileNamesList, filePath):
    # Given a list of file names and a path for the files, returns a
    # list of file objects
    fObjList = []
    for fileName in fileNamesList:
        fObj = open(filePath + fileName, 'r', encoding='utf8')
        fObjList.append(fObj)
    return fObjList

filesListFree = openFiles(fileNamesFree, textPathFree)
filesListDC = openFiles(fileNamesDC, textPathDC)

# Some of these files have non-UTF characters that may need to be removed.
def removeSpecials(filesList):
    # Remove some incompatible unicode characters such as distinct
    # opening and closing quotes, ellipsis etc.
    # Also finally converts all text to lowercase
    outString = ''
    for inFile in filesList:
        # print("reading file",inFile.name.split("/")[-1], "...", end="")
        temp = inFile.read()
        subsFileQuotes = re.sub(u"(\u2018|\u2019)", "'", temp)
        subsFileDQuotes = re.sub(u'(\u201c|\u201d)', '"', subsFileQuotes)
        subsFileEllipsis = re.sub(u"\u2026", "...", subsFileDQuotes)
        outString += subsFileEllipsis
        # print("done.")
    return outString.lower()

group1_raw = removeSpecials(filesListFree)
group2_raw = removeSpecials(filesListDC)

# These files have the transcriber's observations like [laughter], [silent], [cross-talk] etc.
# These may throw off the text analysis, so we remove them. These are typically within parantheses,
# so we can remove everything between these parantheses.

def removeObservations(test_str) :
    # removes all text within parantheses
    # these are usually observations by the transcriber
    # and thus should not be considered in content analysis
    # code from http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
    ret = ''
    skip1c = 0
    skip2c = 0
    squareparen = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
            squareparen += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

group1 = removeObservations(group1_raw)
group2 = removeObservations(group2_raw)

group1_lines = group1.split('\n')
group2_lines = group2.split('\n')

print("Number of lines in the Free Group:", len(group1_lines))
print("Number of lines in the D-C Group:", len(group2_lines))
print("Each line will be considered a separate document for computing the Document-Term Matrix")

Number of lines in the Free Group: 116
Number of lines in the D-C Group: 110
Each line will be considered a separate document for computing the Document-Term Matrix


# Step 2: Computing the Latent Dirichlet Allocation
Detailed explanation to follow...


In [2]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

## Step 2.1 Computing the Document-Term Matrix

The document-term matrix looks like this:

|      | T1     | T2     | T3     | T4     | T5     |... 
|:----:|:------:|:------:|:------:|:------:|:------:|:---:
| D1   | N_d1t1 | N_d1t2 | N_d1t2 | N_d1t3 | N_d1t4 |...
| D2   | N_d2t1 | N_d2t2 | N_d2t2 | N_d2t3 | N_d2t4 |...
| D3   | N_d3t1 | N_d3t2 | N_d3t2 | N_d3t3 | N_d3t4 |...
| D4   | N_d4t1 | N_d4t2 | N_d4t2 | N_d4t3 | N_d4t4 |...
| D5   | N_d5t1 | N_d5t2 | N_d5t2 | N_d5t3 | N_d5t4 |...
| D6   | N_d6t1 | N_d6t2 | N_d6t2 | N_d6t3 | N_d6t4 |...
| ..   | ...... | ...... | ...... | ...... | ...... |...

Where N_ditj is called the **term frequency**, and is equal to the the number of occurrences of term Tj in document Di.

For our purpose, each term is a word longer than two letters, and each document is a line in the transcript. We need to focus on words that matter, so this means removing any stop words, as well as words from the transcript that we deem irrelevant. These include filler words such as "yeah", "like", etc. and non-words such as "hmm". Since this is a transcript, we find a number of time-stamped entries embedded in the text, so these need to go as well. A complete list of words to remove from the list of terms is below (excluding stop words).

Each document is represented as an M-dimensional vector where M is the number of terms computed for the corpus. Here, the vector corresponds to the row of the document-term matrix.

In [3]:
words_to_remove = ['yea', 'like', 'up', 'down', "!", "$", "'", "''", "'cause", 
                   "'d", "'em", "'ll", "'m", "'re", "'s", "'ve", ",", "yeah",
                   "-", ".", "1", "10", "12", "12.", "15", "20", "3-d", 
                   "30", "300", "352.", "36", "375", "3d", "452", "475", 
                   "475.", "90", ":", ";", "?", "``", "one", "would", "right",
                   "okay", 'could', 'know', 'make', 'something', 'yes', 'good',
                   "think", "oh", "maybe", 'kinda', 'actually', 'thing', 'things',
                   'gonna', 'wanna', 'really', 'hmm']

complete_stop_words = stopwords.words('english') + words_to_remove

tf_vectorizer_grp1 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)

tf_vectorizer_grp2 = CountVectorizer(strip_accents = 'unicode',
                                     analyzer = 'word',
                                     stop_words = complete_stop_words,
                                     lowercase = True,
                                     token_pattern = r'\b[a-zA-Z]{3,}\b',
                                     max_df = 0.5, 
                                     min_df = 10)
dtm_tf_grp1 = tf_vectorizer_grp1.fit_transform(group1_lines)
print("Document-Term Matrix size for Free Group:", dtm_tf_grp1.shape)
dtm_tf_grp2 = tf_vectorizer_grp2.fit_transform(group2_lines)
print("Document-Term Matrix size for D-C Group:", dtm_tf_grp2.shape)

Document-Term Matrix size for Free Group: (116, 56)
Document-Term Matrix size for D-C Group: (110, 55)


## 2.2 Compute the Term Frequency - Inverse Document Frequency
In order to use a scheme for weighting the document-term matrix, we use the popular **Inverse Document Frequency (IDF)** weighting measure.
The IDF is calculated as:

$IDF = log(N / n_j)$

Where $N$ is the total number of documents, and $n_j$ is the number of documents containing term $j$.
This system gives higher weightage to terms that occur in relatively fewer documents (and thus are more characteristic of the *topic* of that document)

The weighted matrix is called Term Frequency-Inverse Document Frequency, or **Tf-idf**, where each cell (term frequency) is multiplied by the weight (inverse document frequency)

In [4]:
tfidf_vectorizer_grp1 = TfidfVectorizer(**tf_vectorizer_grp1.get_params())
dtm_tfidf_grp1 = tfidf_vectorizer_grp1.fit_transform(group1_lines)

tfidf_vectorizer_grp2 = TfidfVectorizer(**tf_vectorizer_grp2.get_params())
dtm_tfidf_grp2 = tfidf_vectorizer_grp2.fit_transform(group2_lines)

## 2.3 Compute Latent Dirichlet Allocation (LDA)
The following explanation is from [Edwin Chen's website](http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/)

LDA represents documents as mixtures of topics that spit out words with certain probabilities. It assumes that documents are produced in the following fashion: when writing each document, you:

* Decide on the number of words N the document will have (say, according to a Poisson distribution).
* Choose a topic mixture for the document (according to a Dirichlet distribution over a fixed set of K topics). For example, assuming that we have the two food and cute animal topics above, you might choose the document to consist of 1/3 food and 2/3 cute animals.
* Generate each word w_i in the document by:
    * First picking a topic (according to the multinomial distribution that you sampled above; for example, you might pick the food topic with 1/3 probability and the cute animals topic with 2/3 probability).
    * Using the topic to generate the word itself (according to the topic’s multinomial distribution). For example, if we selected the food topic, we might generate the word “broccoli” with 30% probability, “bananas” with 15% probability, and so on.
* Assuming this generative model for a collection of documents, LDA then tries to backtrack from the documents to find a set of topics that are likely to have generated the collection.

### Example

Let’s make an example. According to the above process, when generating some particular document D, you might

* Pick 5 to be the number of words in D.
* Decide that D will be 1/2 about food and 1/2 about cute animals. (Number of topics = 2)
* Pick the first word to come from the food topic, which then gives you the word “broccoli”.
* Pick the second word to come from the cute animals topic, which gives you “panda”.
* Pick the third word to come from the cute animals topic, giving you “adorable”.
* Pick the fourth word to come from the food topic, giving you “cherries”.
* Pick the fifth word to come from the food topic, giving you “eating”.
So the document generated under the LDA model will be “broccoli panda adorable cherries eating” (note that LDA is a bag-of-words model).


In [5]:
# for TF DTM
lda_tf_grp1 = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tf_grp1.fit(dtm_tf_grp1)
lda_tf_grp2 = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tf_grp2.fit(dtm_tf_grp2)
# for TFIDF DTM
lda_tfidf_grp1 = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tfidf_grp1.fit(dtm_tfidf_grp1)
lda_tfidf_grp2 = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tfidf_grp2.fit(dtm_tfidf_grp2)

C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
C:\Anaconda3\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=10, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

## 2.4 Visualize Free Brainstorming Topics Using Term Frequencies (unweighted)
Using PyLDAvis, visualize topics and salient terms. To focus on terms that are more salient to each topic, set $\lambda$ to 0.4.

In [6]:
pyLDAvis.sklearn.prepare(lda_tf_grp1, dtm_tf_grp1, tf_vectorizer_grp1)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      27.578466        1       1 -0.195791 -0.019031
5      19.285375        1       2  0.108827 -0.030275
9      14.780763        1       3  0.063662  0.016704
4      11.803065        1       4 -0.063536 -0.005131
1       9.601919        1       5  0.075866  0.000355
7       8.391266        1       6  0.029474  0.136988
6       3.550530        1       7  0.034497 -0.023891
2       2.261042        1       8  0.049233 -0.085180
3       2.167893        1       9 -0.066160  0.017484
8       0.579681        1      10 -0.036071 -0.008024, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
28    Default  143.000000          ideas  143.000000  30.0000  30.0000
48    Default  117.000000            toy  117.000000  29.0000  29.0000
43    Default   50.000000        process   50.000000  28.0000  28.0000
11    Default   50.000000       concepts   50.000000  27.0000  27.0000
12    Default   35.000000         design   35.000000  26.0000  26.0000
27    Default   71.000000           idea   71.000000  25.0000  25.0000
46    Default   73.000000           team   73.000000  24.0000  24.0000
47    Default   20.000000           time   20.000000  23.0000  23.0000
2     Default   41.000000           also   41.000000  22.0000  22.0000
54    Default   25.000000          value   25.000000  21.0000  21.0000
21    Default   31.000000          final   31.000000  20.0000  20.0000
49    Default   20.000000           toys   20.000000  19.0000  19.0000
41    Default   41.000000           play   41.000000  18.0000  18.0000
10    Default   87.000000        concept   87.000000  17.0000  17.0000
24    Default   22.000000      generated   22.000000  16.0000  16.0000
25    Default   32.000000     generation   32.000000  15.0000  15.0000
16    Default   20.000000      effective   20.000000  14.0000  14.0000
30    Default   19.000000        initial   19.000000  13.0000  13.0000
19    Default   14.000000        fantasy   14.000000  12.0000  12.0000
18    Default   16.000000           even   16.000000  11.0000  11.0000
44    Default   21.000000        session   21.000000  10.0000  10.0000
37    Default   12.000000           much   12.000000   9.0000   9.0000
6     Default   21.000000  brainstorming   21.000000   8.0000   8.0000
4     Default   13.000000         aspect   13.000000   7.0000   7.0000
31    Default   16.000000           made   16.000000   6.0000   6.0000
8     Default   15.000000       children   15.000000   5.0000   5.0000
35    Default   24.000000         motion   24.000000   4.0000   4.0000
20    Default   24.000000       features   24.000000   3.0000   3.0000
0     Default   23.000000           able   23.000000   2.0000   2.0000
15    Default   27.000000      different   27.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
36    Topic10    0.150461           move   13.711436   0.6382  -4.1031
34    Topic10    0.163031      mechanism   15.297395   0.6090  -4.0228
22    Topic10    0.179857          first   16.897733   0.6077  -3.9246
40    Topic10    0.162193          phase   16.138638   0.5503  -4.0280
4     Topic10    0.133223         aspect   13.355134   0.5428  -4.2247
39    Topic10    0.153564          order   15.434790   0.5402  -4.0827
50    Topic10    0.168001            use   17.330761   0.5142  -3.9928
13    Topic10    0.150359        develop   15.644921   0.5056  -4.1037
18    Topic10    0.153741           even   16.062554   0.5015  -4.0815
7     Topic10    0.143370           came   14.988080   0.5009  -4.1513
24    Topic10    0.210002      generated   22.469706   0.4776  -3.7697
53    Topic10    0.173004          using   18.710492   0.4669  -3.9635
0     Topic10    0.215067           able   23.607678   0.4521  -3.7458
8     Topic10    0.143272       children   

In [7]:
pyLDAvis.sklearn.prepare(lda_tf_grp2, dtm_tf_grp2, tf_vectorizer_grp2)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      23.101714        1       1 -0.104197 -0.013045
0      21.515404        1       2  0.131216 -0.005390
8      20.335273        1       3 -0.177113  0.044688
5      11.757049        1       4  0.066754  0.109913
7       7.290723        1       5  0.122856 -0.066571
1       6.420037        1       6  0.009324 -0.026805
2       5.493298        1       7 -0.092169 -0.073494
4       2.806792        1       8  0.036972  0.043918
3       0.639857        1       9  0.004471 -0.007080
6       0.639854        1      10  0.001886 -0.006135, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
26    Default  148.000000       ideas  148.000000  30.0000  30.0000
36    Default   48.000000        play   48.000000  29.0000  29.0000
46    Default   95.000000         toy   95.000000  28.0000  28.0000
44    Default   42.000000        team   42.000000  27.0000  27.0000
20    Default   29.000000        game   29.000000  26.0000  26.0000
17    Default   25.000000    features   25.000000  25.0000  25.0000
37    Default   40.000000     process   40.000000  24.0000  24.0000
25    Default   73.000000        idea   73.000000  23.0000  23.0000
2     Default   54.000000        also   54.000000  22.0000  22.0000
18    Default   47.000000       final   47.000000  21.0000  21.0000
10    Default   39.000000      design   39.000000  20.0000  20.0000
33    Default   23.000000      motion   23.000000  19.0000  19.0000
16    Default   21.000000     feature   21.000000  18.0000  18.0000
45    Default   23.000000        time   23.000000  17.0000  17.0000
6     Default   26.000000         car   26.000000  16.0000  16.0000
22    Default   32.000000  generation   32.000000  15.0000  15.0000
53    Default   20.000000       value   20.000000  14.0000  14.0000
23    Default   28.000000       group   28.000000  13.0000  13.0000
50    Default   13.000000         use   13.000000  12.0000  12.0000
12    Default   23.000000   developed   23.000000  11.0000  11.0000
52    Default   23.000000        user   23.000000  10.0000  10.0000
5     Default   20.000000        came   20.000000   9.0000   9.0000
54    Default   22.000000        well   22.000000   8.0000   8.0000
7     Default   22.000000        come   22.000000   7.0000   7.0000
1     Default   16.000000     allowed   16.000000   6.0000   6.0000
19    Default   18.000000       first   18.000000   5.0000   5.0000
3     Default   16.000000     another   16.000000   4.0000   4.0000
8     Default   78.000000    concepts   78.000000   3.0000   3.0000
32    Default   18.000000         may   18.000000   2.0000   2.0000
39    Default   16.000000     session   16.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
11    Topic10    0.170442     develop   13.648882   0.6687  -4.0342
42    Topic10    0.172170     started   14.064803   0.6487  -4.0241
3     Topic10    0.183368     another   16.048758   0.5798  -3.9611
0     Topic10    0.175025        able   15.503915   0.5678  -4.0077
1     Topic10    0.183446     allowed   16.951834   0.5255  -3.9607
21    Topic10    0.162892   generated   15.492174   0.4967  -4.0795
9     Topic10    0.183085     decided   17.599363   0.4860  -3.9627
40    Topic10    0.166332     several   16.140444   0.4766  -4.0586
34    Topic10    0.181402        much   17.835766   0.4634  -3.9719
39    Topic10    0.166178     session   16.357317   0.4623  -4.0596
32    Topic10    0.191184         may   18.924397   0.4567  -3.9194
38    Topic10    0.173122     product   18.288532   0.3917  -4.0186
47    Topic10    0.166569        toys   18.070327   0.3651  -4.0572
19    Topic10    0.172980       first   18.808608   0.3628  -4.0194
35    Topic10    0.150664       phase   16.855116   0.3343  -4.1576
5     Topic10    0.182587        came   20.76

In [8]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp1, dtm_tfidf_grp1, tfidf_vectorizer_grp1)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      36.339124        1       1 -0.089093  0.010300
0      21.418489        1       2  0.074137  0.031788
4      14.721933        1       3  0.075947 -0.040415
2       7.186143        1       4 -0.045294 -0.044102
9       6.355637        1       5 -0.008865  0.041104
5       3.144417        1       6 -0.008617 -0.001409
3       2.844455        1       7  0.009571 -0.001284
8       2.663270        1       8 -0.006949  0.000781
1       2.663269        1       9  0.001136  0.002942
6       2.663264        1      10 -0.001974  0.000296, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
28    Default  13.000000          ideas  13.000000  30.0000  30.0000
48    Default  13.000000            toy  13.000000  29.0000  29.0000
54    Default   5.000000          value   5.000000  28.0000  28.0000
46    Default   9.000000           team   9.000000  27.0000  27.0000
12    Default   6.000000         design   6.000000  26.0000  26.0000
43    Default   7.000000        process   7.000000  25.0000  25.0000
11    Default   8.000000       concepts   8.000000  24.0000  24.0000
8     Default   4.000000       children   4.000000  23.0000  23.0000
52    Default   4.000000           user   4.000000  22.0000  22.0000
23    Default   3.000000           game   3.000000  21.0000  21.0000
41    Default   6.000000           play   6.000000  20.0000  20.0000
0     Default   4.000000           able   4.000000  19.0000  19.0000
25    Default   6.000000     generation   6.000000  18.0000  18.0000
4     Default   3.000000         aspect   3.000000  17.0000  17.0000
21    Default   6.000000          final   6.000000  16.0000  16.0000
27    Default   8.000000           idea   8.000000  15.0000  15.0000
24    Default   5.000000      generated   5.000000  14.0000  14.0000
19    Default   3.000000        fantasy   3.000000  13.0000  13.0000
20    Default   5.000000       features   5.000000  12.0000  12.0000
6     Default   4.000000  brainstorming   4.000000  11.0000  11.0000
32    Default   3.000000           many   3.000000  10.0000  10.0000
47    Default   4.000000           time   4.000000   9.0000   9.0000
31    Default   3.000000           made   3.000000   8.0000   8.0000
26    Default   4.000000          group   4.000000   7.0000   7.0000
16    Default   4.000000      effective   4.000000   6.0000   6.0000
2     Default   6.000000           also   6.000000   5.0000   5.0000
55    Default   4.000000           well   4.000000   4.0000   4.0000
49    Default   4.000000           toys   4.000000   3.0000   3.0000
30    Default   4.000000        initial   4.000000   2.0000   2.0000
9     Default   3.000000           come   3.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
3     Topic10   0.140049        another   4.144495   0.2381  -3.9606
51    Topic10   0.107710           used   3.187660   0.2380  -4.2231
31    Topic10   0.130625           made   3.899634   0.2293  -4.0302
33    Topic10   0.114294            may   3.420001   0.2270  -4.1638
13    Topic10   0.118971        develop   3.580995   0.2211  -4.1237
19    Topic10   0.124053        fantasy   3.757373   0.2149  -4.0819
53    Topic10   0.115529          using   3.575171   0.1934  -4.1531
38    Topic10   0.108236            new   3.361544   0.1898  -4.2183
32    Topic10   0.123196           many   3.837673   0.1868  -4.0888
40    Topic10   0.106428          phase   3.360326   0.1733  -4.2351
34    Topic10   0.122605      mechanism   3.872284   0.1730  -4.0936
29    Topic10   0.103619     individual   3.440353   0.1230  -4.2618
55    Topic10   0.145301           well   4.914194   0.1045  -3.9238
24    Topic10   0.145784      generated   5.114894   0.0678  -3.9204
12    Topic10   0.163330         design   6.234501  -0.0165  -3.80

In [9]:
pyLDAvis.sklearn.prepare(lda_tfidf_grp2, dtm_tfidf_grp2, tfidf_vectorizer_grp2)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      35.878643        1       1 -0.090120  0.003549
1      18.348863        1       2  0.057425 -0.001668
2      14.998725        1       3  0.058036 -0.003613
7      10.338343        1       4  0.009074  0.048067
8       5.305383        1       5  0.010666 -0.024468
5       3.585539        1       6 -0.003629  0.004748
4       3.140490        1       7 -0.022803 -0.013099
6       3.050490        1       8 -0.009833 -0.011313
9       2.676807        1       9  0.000084  0.004214
3       2.676717        1      10 -0.008901 -0.006417, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
26    Default  14.000000          ideas  14.000000  30.0000  30.0000
35    Default   3.000000          phase   3.000000  29.0000  29.0000
25    Default   8.000000           idea   8.000000  28.0000  28.0000
5     Default   4.000000           came   4.000000  27.0000  27.0000
17    Default   6.000000       features   6.000000  26.0000  26.0000
32    Default   4.000000            may   4.000000  25.0000  25.0000
50    Default   4.000000            use   4.000000  24.0000  24.0000
2     Default   7.000000           also   7.000000  23.0000  23.0000
8     Default   9.000000       concepts   9.000000  22.0000  22.0000
1     Default   3.000000        allowed   3.000000  21.0000  21.0000
18    Default   7.000000          final   7.000000  20.0000  20.0000
33    Default   5.000000         motion   5.000000  19.0000  19.0000
51    Default   3.000000           used   3.000000  18.0000  18.0000
16    Default   5.000000        feature   5.000000  17.0000  17.0000
19    Default   3.000000          first   3.000000  16.0000  16.0000
44    Default   6.000000           team   6.000000  15.0000  15.0000
36    Default   7.000000           play   7.000000  14.0000  14.0000
20    Default   7.000000           game   7.000000  13.0000  13.0000
45    Default   4.000000           time   4.000000  12.0000  12.0000
12    Default   5.000000      developed   5.000000  11.0000  11.0000
48    Default   4.000000            two   4.000000  10.0000  10.0000
22    Default   5.000000     generation   5.000000   9.0000   9.0000
52    Default   5.000000           user   5.000000   8.0000   8.0000
27    Default   4.000000        initial   4.000000   7.0000   7.0000
6     Default   6.000000            car   6.000000   6.0000   6.0000
46    Default  11.000000            toy  11.000000   5.0000   5.0000
37    Default   6.000000        process   6.000000   4.0000   4.0000
23    Default   5.000000          group   5.000000   3.0000   3.0000
7     Default   4.000000           come   4.000000   2.0000   2.0000
4     Default   3.000000  brainstorming   3.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
19    Topic10   0.141345          first   3.721267   0.3500  -3.9905
51    Topic10   0.145087           used   3.904269   0.3281  -3.9644
41    Topic10   0.133968          since   3.656501   0.3139  -4.0441
40    Topic10   0.145089        several   4.127252   0.2726  -3.9644
3     Topic10   0.145080        another   4.185135   0.2586  -3.9644
35    Topic10   0.135070          phase   3.896666   0.2585  -4.0359
0     Topic10   0.133956           able   3.996905   0.2248  -4.0442
7     Topic10   0.137459           come   4.106969   0.2235  -4.0184
47    Topic10   0.136004           toys   4.140539   0.2047  -4.0290
39    Topic10   0.132331        session   4.095068   0.1883  -4.0564
45    Topic10   0.138464           time   4.295833   0.1858  -4.0111
21    Topic10   0.127657      generated   4.043666   0.1650  -4.0924
50    Topic10   0.136181            use   4.316607   0.1643  -4.0277
34    Topic10   0.135883           much   4.348950   0.1547  -4.0299
5     Topic10   0.134736           came   4.345587   0.1470  -4.03